In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import gspread
from bs4 import BeautifulSoup as bs
import datetime
import pandas as pd
import json
import time


In [11]:
Sev_Day_Date = (datetime.datetime.now() - datetime.timedelta(days = 7)).strftime('%d-%m-%Y')
today_date = datetime.datetime.now().strftime('%d-%m-%Y')

In [12]:
print(f'{Sev_Day_Date} and {today_date}')

07-02-2024 and 14-02-2024


In [13]:
url  = f'https://www.nseindia.com/api/circulars?fromDate={Sev_Day_Date}&toDate={today_date}'
header = {
'Accept':'*/*',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'en-US,en;q=0.6',
'Sec-Ch-Ua':'"Not A(Brand";v="99", "Brave";v="121", "Chromium";v="121"',
'Sec-Ch-Ua-Mobile':'?0',
'Sec-Ch-Ua-Platform':'"Windows"',
'Sec-Fetch-Dest':'empty',
'Sec-Fetch-Mode':'cors',
'Sec-Fetch-Site':'same-origin',
'Sec-Gpc':'1',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
}
chrome_options = Options()
chrome_options.add_argument('--headless')
for key, value in header.items():
    chrome_options.add_argument(f"{key}={value}")
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.nseindia.com/resources/exchange-communication-circulars')
driver.get(url)
page_source = driver.page_source
time.sleep(1)
driver.close()

In [14]:
soup = bs(page_source,'html.parser')
htmlsoup=(soup.find('pre')).string
Data = json.loads(htmlsoup)
temp1 = Data.get('data',[])
Master_Data = pd.DataFrame(temp1)

In [15]:
Master_Data = Master_Data[['cirDisplayDate','sub','circFilelink','circCategory','circDepartment']]

,cirDisplayDate,sub,circFilelink,circCategory,circDepartment
0,"February 14, 2024",Revision of Initial Margin in Silver,https://nsearchives.nseindia.com/content/circu...,Trading,Commodity Derivatives
1,"February 14, 2024",CM - Multiple Settlements Scheduled on Februar...,https://nsearchives.nseindia.com/content/circu...,Clearing,NSE Clearing - Capital Market
2,"February 14, 2024",Revamp of MFSS Platform - Update,https://nsearchives.nseindia.com/content/circu...,Trading,Mutual Fund
3,"February 13, 2024",Corrigendum - Liquidity Enhancement Scheme - S...,https://nsearchives.nseindia.com/content/circu...,Trading,Commodity Derivatives
4,"February 13, 2024",Listing of Equity Shares of Capital Small Fina...,https://nsearchives.nseindia.com/content/circu...,Listing,Listing
5,"February 13, 2024",Listing of Equity Shares of Rashi Peripherals ...,https://nsearchives.nseindia.com/content/circu...,Listing,Listing
6,"February 13, 2024",Listing of further issues of securities,https://nsearchives.nseindia.com/content/circu...,Listing,Listing
7,"February 13, 2024",Listing of further issues of Securities,https://nsearchives.nseindia.com/content/circu...,Listing,Listing
8,"February 13, 2024",Listing of Equity Shares of Jana Small Finance...,https://nsearchives.nseindia.com/content/circu...,Listing,Listing
9,"February 13, 2024",Listing of further issues of Ishan Internation...,https://nsearchives.nseindia.com/content/circu...,Listing,Listing


In [16]:
filtered_df = Master_Data[Master_Data['sub'].str.contains('Listing of Equity|Listing of Sovereign Gold|Listing of units')]

In [18]:
#if page_source != '':
gc = gspread.service_account(filename=r'C:\Users\Ashish Kumar Pal\OneDrive\Desktop\Python\Exchange Related task\NSE_Projtects\creds.json')
spreadsheet_name = 'Notifications and Listings'
sheet_name = 'NSE_Forthcoming_listing' 
sh = gc.open(spreadsheet_name).worksheet(sheet_name)
existing_data_range = sh.range('A2:L' + str(sh.row_count))
for cell in existing_data_range:
    cell.value = ''

sh.update_cells(existing_data_range)

{'spreadsheetId': '1Otk1m6VxGHQ5nKa5z9bNg5ebTEu7w5k6j4dsSLx5I3s',
 'updatedRange': 'NSE_Forthcoming_listing!A2:L988',
 'updatedRows': 987,
 'updatedColumns': 12,
 'updatedCells': 11844}

In [20]:
#if page_source != '':
values_lists =[]
df_dict = filtered_df.to_dict(orient='records')
for data in df_dict:
    values_lists.append([str(data['cirDisplayDate']), str(data['sub']), str(data['circFilelink']),str(data['circCategory']),str(data['circDepartment'])])
sh.append_row([f'Batch ran at: {datetime.datetime.now()}'])
sh.append_rows(values_lists)
#else:
    #pass


{'spreadsheetId': '1Otk1m6VxGHQ5nKa5z9bNg5ebTEu7w5k6j4dsSLx5I3s',
 'tableRange': 'NSE_Forthcoming_listing!A1:C2',
 'updates': {'spreadsheetId': '1Otk1m6VxGHQ5nKa5z9bNg5ebTEu7w5k6j4dsSLx5I3s',
  'updatedRange': 'NSE_Forthcoming_listing!A3:E14',
  'updatedRows': 12,
  'updatedColumns': 5,
  'updatedCells': 60}}

In [21]:
#filter_Data.head(30)

In [22]:
#for i in filter_Data['circFilelink']:
    #print(i)